## Ensemble Learning
- If you aggregate the predictions of a group of predictors (such as classifiers or regressors), you will often get better predictions than with the best individual predictor. 
- A group of predictors is called an ***ensemble***; thus, this technique is called ***ensemble learning***, and an ensemble algorithm is called an ***ensemble method***.
- For example:
    - You can train a group of decision tree classifiers, each on a different random subset of the training set.
    - You can then obtain the predictions of all the individual trees, and the class that gets the most votes is the ensemble's prediction.
    - Such an ensemble of decision trees is called a ***random forest***, and despite its simplicity, this is one of the most powerful machine learning algorithms available today.
- You will often use ensemble methods near the end of a project, once you have already built a few good predictors, to combine them into an even better predictor.

### Voting Classifiers
- Suppose you have trained a few classifiers, each one achieving about 80% accuracy.
- *You may have a logistic regression classifier, an SVM classifier, a random forest classifier, a k-nearest neighbors classifier, and perhaps a few more*.
- A very simple way to create an even better classifier is to aggregate the predictions of each classifier: the class that gets the most votes is the ensemble's prediction.
- This majority-vote classifier is called a *hard-voting* classifier.
- Somewhat surprisingly, this voting classifier often achieves a higher accuracy than the best classifier in the ensemble. 
- In fact, even if each classifier is a *weak learner* (meaning it only does slightly better than random guessing), the ensemble can still be a *strong learner* (achieving high accuracy), provided there are a sufficient number of weak learners in the ensemble and they are sufficiently diverse.

***Ensemble methods work best when the predictors are as independent from one another as possible. One way to get diverse classifiers is to train them using very different algorithms. This increases the chance that they will make very different types of errors, improving the ensemble's accuracy.***

- Scikit-Learn's `VotingClassifier` class:
    - Just give it a list of name/predictor pairs, and use it like a normal classifier.

In [1]:
# Using VotingClassifier on the make_moons dataset
# We will load and split the moons dataset into a training set and a test set, then we'll create and train a voting classifier
# composed of three diverse classifiers


from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42, probability=True))
    ]
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))])

- When you fit a `VotingClassifier`, it clones every estimator and fits the clones.
- The original estimators are available via the `estimators` attribute, while the fitted clones are available via the `estimators_` attribute.

In [2]:
# Let's look at each fitted classifier's accuracy on the test set
for name, clf in voting_clf.named_estimators_.items():
    print(f"{name} = {clf.score(X_test, y_test)}")

lr = 0.864
rf = 0.896
svc = 0.896


In [3]:
# For each classifier, show the prediction of that classifier on the first instance of the test set
for clf in voting_clf.estimators_:
    y_pred = clf.predict(X_test[:1])
    print(y_pred)

[1]
[1]
[0]


In [4]:
# So if we use the predict method of the voting classifier, it should give us the most frequent prediction among the three classifiers
voting_clf.predict(X_test[:1])

array([1])

In [5]:
# Now if we look at the voting classifier's score on the test set, it should be higher than each individual classifier
voting_clf.score(X_test, y_test)

0.912

- If all classifiers are able to estimate class probabilities (i.e. if they all have a `predict_proba()` method), then you can tell Scikit-Learn to predict the class with the highest class probability, averaged over all the individual classifiers.
- This is called *soft voting*. 
- It often achieves higher performance than hard voting because it gives more weight to highly confident votes.

In [6]:
voting_clf.voting = "soft"

In [7]:
# Taking the predicted probabilities of each class from each classifier for the first test instance
import numpy as np

class_0_probs = []
class_1_probs = []
for clf in voting_clf.estimators_:
    class_0_probs.append(clf.predict_proba(X_test[:1])[0][0])
    class_1_probs.append(clf.predict_proba(X_test[:1])[0][1])

class_0_probs = np.array(class_0_probs)
class_1_probs = np.array(class_1_probs)

In [8]:
# Obtaining the mean of the prediction confidence from each class
print(class_0_probs)
print(class_0_probs.mean())
print(class_1_probs)
print(class_1_probs.mean())

[0.49900001 0.48       0.56979485]
0.5162649531151421
[0.50099999 0.52       0.43020515]
0.48373504688485797


In [9]:
# So now with soft voting, we should expect the prediction on the first test instance to be class 0
voting_clf.predict(X_test[:1])

array([0])

In [10]:
# But our overall score on the test set is
voting_clf.score(X_test, y_test)

0.92

### Bagging and Pasting
- Another approach to get very different training algorithms is to use the same training algorithm for every predictor but train them on different random subsets of the training set. 
- When sampling is performed *with* replacement, this method is called ***bagging*** (short for *bootstrap aggregating*). 
- When sampling is performed *without* replacement, this method is called ***pasting***.
- In other words, both bagging and pasting allow training instances to be sampled several times across multiple predictors, but only bagging allows training instances to be sampled several times for the same predictor.
- Once all predictors are trained, the ensemble can make a prediction for a new instance by simply aggregating the predictions of all predictors. 
- The aggregation is typically the *statistical mode* for classification, or the average for regression.
- Predictors can all be trained in parallel, via different CPU cores or even different servers.
- Similary, predictions can be made in parallel.
- This is one of the reasons bagging and pasting are such popular methods: ***they scale very well***.

In [11]:
# The following code trains an ensemble of 500 decision tree classifiers
# Each is trained on 100 training instances randomly sampled from the training set with repalcement
# So this is a bagging example (set bootstrap=False to use pasting)

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500, # The number of classifiers to use
    max_samples=100, # How many instances used to train each classifier
    n_jobs=-1, # use all available cores for training and predictions
    random_state=42
)

In [12]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

***A `BaggingClassifier` automatically performs soft voting instead of hard voting if the base classifier can estimate class probabilities***
- Overall, bagging often results in better models, which explains why it's generally preferred.

### Out-of-Bag Evaluation
- With bagging some training instances may be sampled several times for any given predictor, while others may not be sampled at all
- By default, a `BaggingClassifier` samples $m$ training instances with replacement `(bootstrap=True)`, where $m$ is the size of the training set.
- With this process, it can be shown mathematically that only about 63% of the training instances are sampled on average for each predictor.
- The remaining 37% of the training instances that are not sampled are called *out-of-bag* (OOB) instances. *Note: They are not the same 37% for all predictors*.
- A bagging ensemble can be evaluated using OOB instances, without the need for a separate validation set: indeed, if there are enough estimators, then each instance in the training set will likely be an OOB instance of several estimators, so these estimators can be used to make a fair ensemble prediction for that instance. 
- Once you have a prediction for each instance, you can compute the ensemble's prediction accuracy (or any other metric).

In [13]:
# Using the OOB instances to get a score in sklearn
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    oob_score=True,
    n_jobs=-1,
    random_state=42
)

bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=500,
                  n_jobs=-1, oob_score=True, random_state=42)

In [15]:
bag_clf.oob_score_

0.896

In [16]:
# According to this OOB evaluation, this BaggingClassifier is likely to achieve about 89.6% accuracy on the test set
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.912

### Random Patches and Random Subspaces
- The `BaggingClassifier` class supports sampling the features as well.
- Sampling is controlled by two hyperparameters: `max_features` and `bootstrap_features`. 
- They work the same way as `max_samples` and `bootstrap`, but for feature sampling instead of instance sampling. 
- Thus, each predictor will be trained on a random subset of the input features.
- This technique is particularly useful when you are dealing with high-dimensional inputs (such as images), as it can considerably speed up training.
- Sampling both training instances and features is called the ***random patches*** method.
- Keeping all training instances (by setting `bootstrap=False` and `max_samples=1.0`) but sampling features (by setting `bootstrap_features` to `True` and/or `max_features` to a value smaller than `1.0`) is called the *random subspaces* method.

### Random Forests
- A random forest is an ensemble of decision trees, generally trained via the bagging method, typically with `max_samples` set to the size of the training set.
- Instead of building a `BaggingClassifier` and passing it a `DecisionTreeClassifier`, you can use the `RandomForestClassifier` class, which is more convenient and optimized for decision trees.

In [17]:
# Training a random forest classifier with 500 trees, each limited to a maximum number of 16 leaf nodes
# using all available CPU cores
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)

rnd_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [19]:
y_pred_rf = rnd_clf.predict(X_test) 
accuracy_score(y_pred_rf, y_test)

0.912

- The Random Forest algorithm introduces extra randomness when growing trees; instead of searching for the very best feature when splitting a node, it searches for the best feature among a random subset of features.
- By default, it samples $\sqrt{n}$ features (where $n$ is the total number of features).

### Extra Trees
- When you are growing a tree in a random forest, at each node only a random subset of the features is considered for splitting. 
- It is possible to make trees even more random by also using random thresholds for each feature rather than searching for the best possible thresholds. 
- A forest of such extremely random trees is called an *extremely randomized trees* (or *extra-trees* for short) ensemble.
- Once again, this technique trades more bias for a lower variance.
- It also makes extra-trees classifiers much faster to train than regular random forests, because finding the best possible threshold for each feature at every node is one of the most time-consuming tasks of growing a tree.

***It is hard to tell beforehand whether a `RandomForestClassifier` will perform better or worse than an `ExtraTreesClassifier`. Generally, the only way to know is to try both and compare them using cross-validation.*** 

### Feature Importance
- Random forests make it easy to measure the relative importance of each feature.
- Scikit Learn measures a feature's importance by looking at how much the tree nodes that use that feature reduce impurity on average, across all trees in the forest.
- Scikit-Learn computes this score automatically for each feature after training, then it scales the results so that the sum of all importances is equal to 1.
- You can access the result using the `feature_importances_` variable.